# Ch07. 마트 매출 데이터로 피벗된 데이터를 언피벗하기

## 마트매출 데이터 언피벗해서 집계하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [9]:
url = '../data/Ch07_unpivot.xlsx'

In [20]:
years = ['2020년', '2021년', '2022년']
df_ex1 = pd.DataFrame()
for year in years:
    df = pd.read_excel(url, sheet_name=year)
    df = df.melt('제품', var_name='지점', value_name='매출')
    df['year'] = year
    df_ex1 = pd.concat([df_ex1, df])
df_ex1

,제품,지점,매출,year
0,종합어묵 (300g),강남점,1930000,2020년
1,유기농떡튀밥 (40g),강남점,320000,2020년
2,염장다시마 (400g),강남점,1310000,2020년
3,진시래기국 (500g),강남점,500000,2020년
4,리얼칩스우엉 (30g),강남점,480000,2020년
...,...,...,...,...
2395,매생이 (100g),옥길점,1180000,2022년
2396,찹쌀산자 (120g),옥길점,1290000,2022년
2397,생선가스 (300g),옥길점,550000,2022년
2398,게맛살 (178g),옥길점,1340000,2022년


In [21]:
df_m1 = pd.read_excel(url, sheet_name='지점구분')
df_m1

,지점,지역,분류
0,강남점,서울,직영점
1,서초점,서울,직영점
2,송파점,서울,비직영점
3,노원점,서울,비직영점
4,은평점,서울,비직영점
5,수지점,경기,비직영점
6,영통점,경기,직영점
7,주안점,인천,비직영점
8,검단점,인천,직영점
9,분당점,경기,비직영점


In [22]:
df_m2 = pd.read_excel(url, sheet_name='제품구분')
df_m2

,제품,구분
0,종합어묵 (300g),기타
1,유기농떡튀밥 (40g),기타
2,염장다시마 (400g),기타
3,진시래기국 (500g),기타
4,리얼칩스우엉 (30g),기타
...,...,...
195,매생이 (100g),어패
196,찹쌀산자 (120g),기타
197,생선가스 (300g),기타
198,게맛살 (178g),기타


In [23]:
df_ex = (df_ex1
 .merge(df_m1, on='지점', how='left')
 .merge(df_m2, on='제품', how='left')
 )
df_ex

,제품,지점,매출,year,지역,분류,구분
0,종합어묵 (300g),강남점,1930000,2020년,서울,직영점,기타
1,유기농떡튀밥 (40g),강남점,320000,2020년,서울,직영점,기타
2,염장다시마 (400g),강남점,1310000,2020년,서울,직영점,기타
3,진시래기국 (500g),강남점,500000,2020년,서울,직영점,기타
4,리얼칩스우엉 (30g),강남점,480000,2020년,서울,직영점,기타
...,...,...,...,...,...,...,...
7195,매생이 (100g),옥길점,1180000,2022년,경기,직영점,어패
7196,찹쌀산자 (120g),옥길점,1290000,2022년,경기,직영점,기타
7197,생선가스 (300g),옥길점,550000,2022년,경기,직영점,기타
7198,게맛살 (178g),옥길점,1340000,2022년,경기,직영점,기타


In [25]:
(df_ex
 .pivot_table('매출', index='year', columns='지역', aggfunc='sum')
 .div(100_000_000)
 )

지역,경기,서울,인천
year,,,
2020년,8.7155,10.9754,6.5449
2021년,8.7908,11.1760,6.7754
2022년,8.6829,11.0173,6.6009


In [26]:
(df_ex
 .pivot_table('매출', index='지역', columns='분류', aggfunc='sum')
 .div(100_000_000)
 )

분류,비직영점,직영점
지역,,
경기,12.9938,13.1954
서울,20.0434,13.1253
인천,13.3646,6.5566


In [29]:
(df_ex[df_ex['구분']!='기타']
 .pivot_table('매출', index='구분', columns='year', aggfunc='sum')
 .div(100_000_000)
 )

year,2020년,2021년,2022년
구분,,,
쌀,1.4495,1.4879,1.4609
어패,2.1228,2.1179,2.0343
정육,0.3943,0.4229,0.4530
청과,1.1882,1.1890,1.1023
